<a href="https://colab.research.google.com/github/Hangowoon/Newjak_ex/blob/main/ML03_trainNtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **(1) import libraries**

In [ ]:
#넘파이 numpy 판다스 pandas
import numpy as np
import pandas as pd

# **(2) upload file**

In [ ]:
# nbasta2022.csv를 사용
from google.colab import files 
files.upload()
nbastat = pd.read_csv('nbastat2022.csv')
nbastat

# **(3) Variable 설정**

In [ ]:
# 이전 : 단변량 회귀 : FGA -> PTS
# 이번 : 다면량 회귀 : FGA, 3PA, FTA -> PTS
# FGA -> x1, 3PA -> x2, FTA -> x3 , PTS -> y 

#data의 갯수 : m --> 읽어들인 csv 파일의 줄 수 
m = len(nbastat)

X1 = nbastat[['FGA']]
X2 = nbastat[['3PA']]
X3 = nbastat[['FTA']]
Y = nbastat[['PTS']]

#결측값 제거 --> fillna
X1 = X1.fillna(0)
X2 = X2.fillna(0)
X3 = X3.fillna(0)
Y = Y.fillna(0)

print(type(Y))

# **(4) Array로 변환**

In [ ]:
# pandas의 dataframe형태로 저장된 X1 - Y --> numpy의 array로 변환

X1 = (np.array(X1)).reshape(m,1)
X2 = (np.array(X2)).reshape(m,1)
X3 = (np.array(X3)).reshape(m,1)
Y = (np.array(Y)).reshape(m,1)

print(type(Y))

# **(5) Tensor 변환**

In [ ]:
# Y = theta0 + theta1 * X1 + theta2 * X2 + theta3 * X3
# Y = (theta0, theta1, theta2, theta) * (1, X1, X2, X3)
# (1, X1, X2, X3) -> Xb

X0 = np.ones((m, 1)) #ones는 1을 만들어달라라는 함수, (m,1) m개의 줄을 1을 만들어 달라
Xb = np.c_[X0, X1, X2, X3] # 1과 X를 붙여주세요 --> c_

print(Xb.shape)  
print(Y.shape)

# **(6) dataset 분할**

**(6-1) dataset 분할 함수 정의**

In [ ]:
# X, Y -> 6:4로 분할 Xa, Ya (60%), Xb, Yb (40%)
# Xa, Ya -> training
# Xb, Yb를 적용해서 결과를 비교 

def permutation_split (X, Y, ratio = 0.6, random_state = 100):
  # Xa, Ya에 포함될 데이터의 수 = m, shape[0] * ratio
  # Xb, Yb에 포함될 데이터의 수 = m - Xa에 포함될 데이터의 수 
  num_a = int(m * ratio)
  num_b = m - num_a
  print(num_a, num_b)

  # X에 포함된 data는 m개 -> 0번째부터 m-1번째까지  -> random하게 순서를 바꿉니다
  np.random.seed (random_state)
  shuffle = np.random.permutation(m)

  X = X[shuffle, :]
  Y = Y[shuffle, :]
  Xa = X[:num_a]  # 첫번째 원소부터 num_a개를 저장(0 ~ num_a -1)
  Ya = Y[:num_a]
  Xb = X[num_a:]  # (num_a번째 원소부터 마지막 원소까지 저장)
  Yb = Y[num_a:]

  return Xa, Xb, Ya, Yb 

(6-2) dataset 분할

In [ ]:
Xb_train, Xb_test, Y_train, Y_test = permutation_split(Xb, Y, 0.6)
print(Y_train.shape, Y_test.shape)

# **(7) Gradient descent 변수 설정**

In [ ]:
learning_rate = 0.0001 #alpha : learning rate 
n_iter = 400

# theta = (theta0, theta1, theta2, theta3)
theta = np.zeros((4, 1))
gradient = np.zeros((4,1))

# **(8) Loss 함수 정의**

In [ ]:
def Compute_J (theta, Xb, Y):
  #loss_term = 예측한 값 - 실제 값
  #예측한 값 = theta * X -> Xb.ot(theta) -> 
  loss_term = Xb.dot(theta) - Y 
  loss_term = loss_term.reshape(-1) # loss_term을 (249, 1) -> (249)

  #loss_term의 제곱 
  loss2 = np.square(loss_term)

  #loss_term의 제곱의 평균
  loss_avg = np.sum(loss2)/(2*m)
  return loss_avg

# **(9) Gradient descent 실행**

In [ ]:
loss_arr = []
iter_arr = []
for i in range(n_iter):
  gradient = (1.0/m)*Xb_train.T.dot(Xb_train.dot(theta)-Y_train)
  theta = theta - learning_rate * gradient
  loss = Compute_J(theta, Xb_train, Y_train)
  iter_arr.append(i)
  loss_arr.append(loss)

# **(10) 결과 확인**

In [ ]:
theta

In [ ]:
# 반복 횟수가 증가함에 따라서 loss 함수가 감소함을 보여준다.
# iter_arr, loss_arr를 그려보자

import matplotlib.pyplot as plt
plt.plot(iter_arr, loss_arr, '.b')
plt.xlabel("iteration")
plt.ylabel("loss")

In [ ]:
# train과 test의 error를 비교

# error: Y_prea - Y_train)^2을 더해서 (1/m), m --> Y_train의 수 또는 Y_test의 수 
# (Y_pred - Y_train)^2을 더한다 --> 내적  --> dot 
# (Y_pred - Y_train).T.dot(Y_pred - Y_train)

# train dataset에 대한 error
Y_pred = Xb_train.dot(theta)
error_train = (1/len(Y_train))*(Y_pred - Y_train).T.dot(Y_pred - Y_train)

# test detaset에 대한 error 
Y_pred = Xb_test.dot(theta)
error_test = (1/len(Y_test))*(Y_pred - Y_test).T.dot(Y_pred - Y_test)

print(error_train, error_test)
print(Y_test)